In [1]:
from ipywidgets import interact

In [2]:
import pandas as pd
import numpy as np

In [3]:
from bokeh.io import show, output_notebook, push_notebook #curdoc, 
from bokeh.plotting import figure

from bokeh.models import CategoricalColorMapper, HoverTool, ColumnDataSource, Panel
from bokeh.models.widgets import CheckboxGroup, Slider, RangeSlider, Tabs, Dropdown, TextInput, Select

from bokeh.layouts import column, row, WidgetBox
from bokeh.palettes import Category20_16

from bokeh.application.handlers import FunctionHandler
from bokeh.application import Application

output_notebook()

Loading BokehJS ...

In [4]:
from io import BytesIO

In [5]:
import zipfile as zf

In [6]:
from urllib.request import urlopen

In [7]:
# url = urlopen("http://databank.worldbank.org/data/download/WDI_csv.zip")
url = 'data/WDI_csv.zip'

In [8]:
# wdi_zip = zf.ZipFile(BytesIO(url.read()))
wdi_zip = zf.ZipFile(url)

In [9]:
wdi_zip.namelist()

['WDIData.csv',
 'WDICountry.csv',
 'WDISeries.csv',
 'WDICountry-Series.csv',
 'WDISeries-Time.csv',
 'WDIFootNote.csv']

In [10]:
df = pd.read_csv(wdi_zip.open('WDIData.csv'))

In [11]:
df.shape

(420024, 63)

In [12]:
df

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,Unnamed: 62
0,Arab World,ARB,"2005 PPP conversion factor, GDP (LCU per inter...",PA.NUS.PPP.05,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Arab World,ARB,"2005 PPP conversion factor, private consumptio...",PA.NUS.PRVT.PP.05,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Arab World,ARB,Access to clean fuels and technologies for coo...,EG.CFT.ACCS.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,8.184498e+01,8.240765e+01,8.282764e+01,8.316923e+01,8.358714e+01,8.395429e+01,8.423063e+01,8.457043e+01,NaN,NaN
3,Arab World,ARB,Access to electricity (% of population),EG.ELC.ACCS.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,8.518982e+01,8.613613e+01,8.678268e+01,8.728824e+01,8.838971e+01,8.807677e+01,8.851797e+01,8.876865e+01,NaN,NaN
4,Arab World,ARB,"Access to electricity, rural (% of rural popul...",EG.ELC.ACCS.RU.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,7.354170e+01,7.450747e+01,7.565271e+01,7.662832e+01,7.866374e+01,7.743907e+01,7.835552e+01,7.874321e+01,NaN,NaN
5,Arab World,ARB,"Access to electricity, urban (% of urban popul...",EG.ELC.ACCS.UR.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,9.508829e+01,9.584153e+01,9.603310e+01,9.606063e+01,9.644658e+01,9.648123e+01,9.664141e+01,9.677328e+01,NaN,NaN
6,Arab World,ARB,Account ownership at a financial institution o...,FX.OWN.TOTL.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,2.226054e+01,NaN,NaN,3.027713e+01,NaN,NaN,37.165211,NaN
7,Arab World,ARB,Account ownership at a financial institution o...,FX.OWN.TOTL.FE.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1.377582e+01,NaN,NaN,2.207935e+01,NaN,NaN,25.635403,NaN
8,Arab World,ARB,Account ownership at a financial institution o...,FX.OWN.TOTL.MA.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,3.037767e+01,NaN,NaN,3.779076e+01,NaN,NaN,48.328518,NaN
9,Arab World,ARB,Account ownership at a financial institution o...,FX.OWN.TOTL.OL.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,2.574129e+01,NaN,NaN,3.421658e+01,NaN,NaN,42.542046,NaN


In [13]:
available_countries = list(df['Country Name'].unique())
available_countries.sort()

In [14]:
available_indicators = list(df['Indicator Code'].unique())
available_indicators.sort()

In [41]:
def modify_doc(doc):
    def make_dataset(country, indicator):
        by_country = df[(df['Country Name'] == country) & (df['Indicator Code'] == indicator)].T
        indicator_name = by_country[2:3]
        indicator_name = indicator_name.T.reset_index()['Indicator Name']
        indicator_name = indicator_name[0]
    
        series_data = by_country[4:62]
    
        series_data = series_data.reset_index()
        series_data.columns = ['Year', 'value']
        
        return ColumnDataSource(indicator_name), ColumnDataSource(series_data)
    
    def style(p):
        # Title 
        p.title.align = 'center'
        p.title.text_font_size = '20pt'
        p.title.text_font = 'serif'

        # Axis titles
        p.xaxis.axis_label_text_font_size = '14pt'
        p.xaxis.axis_label_text_font_style = 'bold'
        p.yaxis.axis_label_text_font_size = '14pt'
        p.yaxis.axis_label_text_font_style = 'bold'

        # Tick labels
        p.xaxis.major_label_text_font_size = '12pt'
        p.yaxis.major_label_text_font_size = '12pt'

        return p
    
    def make_plot(src, indicator_name):
        # Blank plot with correct labels
        p = figure(plot_width = 700, plot_height = 700, 
                  title = indicator_name,
                  x_axis_label = 'Year', y_axis_label = 'value')
        p.line('Year', 'value', source=src)
        p = style(p)
        return p
    
    def update(attr, old, new):
        # Get the list of carriers for the graph
        # country_to_plot = [country_selection.labels[i] for i in 
        #                    country_selection.active]
        #indicator_to_plot = [indicator_selection.labels[i] for i in indicator_selection.active]
        country_to_plot = country_selection.value
        indicator_to_plot = indicator_selection.value
        # Make a new dataset based on the selected carriers and the 
        # make_dataset function defined earlier
        new_indicator_name, new_src = make_dataset(country_to_plot, indicator_to_plot)

        # Convert dataframe to column data source
        # = ColumnDataSource(new_src)

        # Update the source used the quad glpyhs
        src.data.update(new_src.data)
        indicator_name.data.update(new_indicator_name.data)
    
    country_selection = Select(options=available_countries, value='Australia',title='Country')
    country_selection.on_change('value', update)
    indicator_selection = Select(options=available_indicators, value='SP.URB.TOTL',title='Indicators')
    indicator_selection.on_change('value', update)
    
    country = 'Australia'
    indicator = 'SP.URB.TOTL'
    #by_country = make_dataset(country, indicator)
    
    
    indicator_name, src = make_dataset(country, indicator)
    p = make_plot(src, indicator_name)
    selection_layout = row(country_selection, indicator_selection)
    layout = column(selection_layout,p)
    doc.add_root(layout)

In [42]:
# Set up an application
handler = FunctionHandler(modify_doc)
app = Application(handler)

In [43]:
show(app, "localhost:8888")

ERROR:bokeh.server.views.ws:received invalid integer in pong b''
Traceback (most recent call last):
  File "C:\Users\Narayanan\Anaconda3\lib\site-packages\bokeh\server\views\ws.py", line 161, in on_pong
    self.latest_pong = int(codecs.decode(data, 'utf-8'))
ValueError: invalid literal for int() with base 10: ''
ERROR:bokeh.server.views.ws:received invalid integer in pong b''
Traceback (most recent call last):
  File "C:\Users\Narayanan\Anaconda3\lib\site-packages\bokeh\server\views\ws.py", line 161, in on_pong
    self.latest_pong = int(codecs.decode(data, 'utf-8'))
ValueError: invalid literal for int() with base 10: ''
ERROR:bokeh.server.views.ws:received invalid integer in pong b''
Traceback (most recent call last):
  File "C:\Users\Narayanan\Anaconda3\lib\site-packages\bokeh\server\views\ws.py", line 161, in on_pong
    self.latest_pong = int(codecs.decode(data, 'utf-8'))
ValueError: invalid literal for int() with base 10: ''
ERROR:bokeh.server.views.ws:received invalid integer in

In [91]:
country = 'Australia'
indicator = 'SP.URB.TOTL'
by_country = make_dataset(country, indicator)

In [107]:
by_country

,92248
Country Name,Australia
Country Code,AUS
Indicator Name,Urban population
Indicator Code,SP.URB.TOTL
1960,8.37831e+06
1961,8.58988e+06
1962,8.84464e+06
1963,9.05861e+06
1964,9.28089e+06
1965,9.50727e+06


In [174]:
indicator_name = by_country[2:3]
indicator_name = indicator_name.T.reset_index()['Indicator Name']
indicator_name = indicator_name[0]

In [179]:
series_data = by_country[4:62]

In [180]:
series_data = series_data.reset_index()
series_data.columns = ['Year', indicator_name]

In [183]:
p = make_plot(ColumnDataSource(series_data), series_data.columns)

In [191]:
def form_menu_group(list):
    return [(x,x) for x in list]

In [209]:
def country_change_handler(attr, old, new):
    print (country_selection.value)

In [211]:
def indicator_change_handler(attr, old, new):
    print(indicator_selection.value)

In [192]:
form_menu_group(available_countries)

[('Afghanistan', 'Afghanistan'),
 ('Albania', 'Albania'),
 ('Algeria', 'Algeria'),
 ('American Samoa', 'American Samoa'),
 ('Andorra', 'Andorra'),
 ('Angola', 'Angola'),
 ('Antigua and Barbuda', 'Antigua and Barbuda'),
 ('Arab World', 'Arab World'),
 ('Argentina', 'Argentina'),
 ('Armenia', 'Armenia'),
 ('Aruba', 'Aruba'),
 ('Australia', 'Australia'),
 ('Austria', 'Austria'),
 ('Azerbaijan', 'Azerbaijan'),
 ('Bahamas, The', 'Bahamas, The'),
 ('Bahrain', 'Bahrain'),
 ('Bangladesh', 'Bangladesh'),
 ('Barbados', 'Barbados'),
 ('Belarus', 'Belarus'),
 ('Belgium', 'Belgium'),
 ('Belize', 'Belize'),
 ('Benin', 'Benin'),
 ('Bermuda', 'Bermuda'),
 ('Bhutan', 'Bhutan'),
 ('Bolivia', 'Bolivia'),
 ('Bosnia and Herzegovina', 'Bosnia and Herzegovina'),
 ('Botswana', 'Botswana'),
 ('Brazil', 'Brazil'),
 ('British Virgin Islands', 'British Virgin Islands'),
 ('Brunei Darussalam', 'Brunei Darussalam'),
 ('Bulgaria', 'Bulgaria'),
 ('Burkina Faso', 'Burkina Faso'),
 ('Burundi', 'Burundi'),
 ('Cabo Verde

In [224]:
#country_selection = CheckboxGroup(labels=available_countries, active = [0])
#country_selection = Dropdown(label="Country", button_type="success", menu = form_menu_group(available_countries))
#country_selection.on_change('value', country_change_handler)
#country_selection.on_click(country_change_handler)
#indicator_selection = CheckboxGroup(labels=available_indicators, active = [0])
#indicator_selection.on_change('value',indicator_change_handler)
#text_input = TextInput(value="default", title="Label:")
#show(WidgetBox(country_selection, indicator_selection, text_input))

show(layout)

In [186]:
[country_selection.labels[i] for i in country_selection.active]

['Afghanistan']